In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
architecture = "MNASNET1_preclinic_v1"
model_name = f"{architecture}"#f"sam_densenet_v1_{label}"

In [3]:
image_size = (224, 224)
bs = 128
lr = 1e-4
epochs = 50
patience = 1
rho = 0.05
with_reset = False
device = get_device()

Using the GPU!


In [4]:
labels = get_labels()
label = PATHOLOGIC

In [5]:
train_df, valid_df, test_df = get_dataframes(include_labels=labels, 
                                                 small=False)

In [6]:
train_df = get_preclinic_df(train_df)
valid_df = get_preclinic_df(valid_df)
test_df = get_preclinic_df(test_df)

In [7]:
train_label = train_df[[label]].values
neg_weights, pos_weights = compute_class_freqs(train_label)
neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)
print(neg_weights, pos_weights)

tensor([0.5862]) tensor([0.4138])


In [8]:
train_tfs, test_tfs = get_transforms(image_size=image_size)

In [9]:
train_ds = CRX8_Data(train_df, get_image_path(), label, image_size=image_size, transforms=train_tfs)
valid_ds = CRX8_Data(valid_df, get_image_path(), label, image_size=image_size, transforms=test_tfs)
test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)

In [10]:
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
dataloaders = {
    "train": train_dl,
    "val": valid_dl,
    "test": test_dl
}

In [11]:
model = pretrained_mnasnet0_5()
model = model.to(device)

In [12]:
criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                          neg_weights.to(device))
sam_optimizer = SAM(model.parameters(), torch.optim.Adam, lr=lr, rho=rho)

In [13]:
model, history = fit(model, criterion, sam_optimizer, 
                     dataloaders, model_name, epochs, 
                     lr, sam=True, with_reset=with_reset, 
                     metric="loss", patience=patience)

Epoch 1:



Train: Loss: 0.322, Acc: 0.622, AUROC: 0.657



Val: Loss: 0.404, Acc: 0.654, AUROC: 0.695
Saved model with loss 0.4043
Epoch 2:



Train: Loss: 0.301, Acc: 0.664, AUROC: 0.717



Val: Loss: 0.368, Acc: 0.671, AUROC: 0.719
Saved model with loss 0.3679
Epoch 3:


...


Val: Loss: 0.291, Acc: 0.707, AUROC: 0.751
Lowered lr to 1.0000000000000002e-12
Epoch 28:



Train: Loss: 0.270, Acc: 0.712, AUROC: 0.786



Val: Loss: 0.291, Acc: 0.706, AUROC: 0.751
Lowered lr to 1.0000000000000002e-13
Epoch 29:



Train: Loss: 0.269, Acc: 0.713, AUROC: 0.788



Val: Loss: 0.292, Acc: 0.703, AUROC: 0.751
Lowered lr to 1.0000000000000002e-14
Learning rate is basically zero. Stopping training.


In [14]:
FERTIG()

FERTIG! :D
